# Sentiment analysis (Vader) and Word2Vec for Reddit comment data (r/TheRedPill, 2016)

In [1]:
sc.install_pypi_package("seaborn")
sc.install_pypi_package("pandas")
sc.install_pypi_package("nltk")
sc.install_pypi_package("pandas")
sc.install_pypi_package("pickle")


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1622917956355_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [3]:
sc.install_pypi_package("vader")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Running setup.py bdist_wheel for sonopy: started
  Running setup.py bdist_wheel for sonopy: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/b6/39/ba/b2f21d4fbcb362658c73f83c9502782300b0399aef3693b506
Successfully built sonopy

In [4]:
sc.install_pypi_package("gensim")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version  
-------------------------- ---------
beautifulsoup4             4.9.3    
boto                       2.49.0   
click                      7.1.2    
cycler                     0.10.0   
gensim                     4.0.1    
jmespath                   0.10.0   
joblib                     1.0.1    
kiwisolver                 1.3.1    
lxml                       4.6.2    
matplotlib                 3.4.2    
mysqlclient                1.4.2    
nltk                       3.5      
nose                       1.3.4    
numpy                      1.16.5   
pandas                     1.2.4    
Pillow                     8.2.0    
pip                        9.0.1    
py-dateutil                2.2      
pyparsing                  2.4.7    
python-dateutil            2.8.1    
python37-sagemaker-pyspark 1.4.1    
pytz                       2021.1   
PyYAML                     5.4.1    
regex                      2021.3.17
scikit-learn               0.24.2   
s

In [64]:
df = spark.read.format("csv").option("header", "true").option("delimiter", "\t").option("mode", "DROPMALFORMED").csv("s3://aws-emr-resources-729741394700-us-east-1/reddit_comments_with_sentiment.tsv")
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- subreddit: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- time created: string (nullable = true)
 |-- cleaned_text: string (nullable = true)
 |-- lemmas: string (nullable = true)
 |-- lemmas_str: string (nullable = true)
 |-- neg: string (nullable = true)
 |-- compound: string (nullable = true)
 |-- pos: string (nullable = true)

In [65]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------+-----+
| subreddit|             comment|        time created|        cleaned_text|              lemmas|          lemmas_str|  neg|compound|  pos|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------+-----+
|TheRedPill|Why the fuck woul...|2016-01-01 00:00:...|fuck would friend...|['fuck', 'would',...|fuck would friend ex|0.402| -0.0772|0.368|
|TheRedPill|Great story full ...|2016-01-01 00:01:...|great story full ...|['great', 'story'...|great story full ...|  0.0|  0.8519|0.482|
|TheRedPill|Brazilian here to...|2016-01-01 00:03:...|brazilian man don...|['brazilian', 'ma...|brazilian man don...|0.042| -0.0772|  0.0|
|TheRedPill|Absolutely the la...|2016-01-01 00:06:...|absolutely last t...|['absolutely', 'l...|absolutely last t...|0.112| -0.4404|  0.0|
|TheRedPill|How would you c

In [72]:
df_text = df.filter(df["lemmas_str"].rlike("[a-zA-Z]+")) # ensuring that only non-numeric data is processed

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
sc.install_pypi_package("vaderSentiment")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import format_number as fmt
from pyspark.sql.types import *

from pyspark.ml.feature import Tokenizer,StopWordsRemover,Word2Vec
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
def calc_sent_score(comment): 
    '''
    Gets compound similarity score for a given comment
    '''
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(comment)
    return float(vs['compound'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
udf_sent_score = udf(calc_sent_score, FloatType())
# Applying sentiment score function above to each comment (within 'lemmas_str', cleaned lemmatized comments)

df_sent_score = df_text.withColumn('sentiment_score', udf_sent_score('lemmas_str'))
df_sent_score.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------+-----+---------------+
| subreddit|             comment|        time created|        cleaned_text|              lemmas|          lemmas_str|  neg|compound|  pos|sentiment_score|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------+-----+---------------+
|TheRedPill|Why the fuck woul...|2016-01-01 00:00:...|fuck would friend...|['fuck', 'would',...|fuck would friend ex|0.402| -0.0772|0.368|        -0.0772|
|TheRedPill|Great story full ...|2016-01-01 00:01:...|great story full ...|['great', 'story'...|great story full ...|  0.0|  0.8519|0.482|         0.8519|
|TheRedPill|Brazilian here to...|2016-01-01 00:03:...|brazilian man don...|['brazilian', 'ma...|brazilian man don...|0.042| -0.0772|  0.0|        -0.0772|
|TheRedPill|Absolutely the la...|2016-01-01 00:06:...|absolutely last 

In [75]:
tokenizer = Tokenizer(inputCol='lemmas_str', outputCol='words')
df_tokenized = tokenizer.transform(df_sent_score) # tokenizing comments (in lemma_str) for word2vec

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
df_tokenized.select('cleaned_text','sentiment_score','words').show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+--------------------+
|        cleaned_text|sentiment_score|               words|
+--------------------+---------------+--------------------+
|fuck would friend...|        -0.0772|[fuck, would, fri...|
|great story full ...|         0.8519|[great, story, fu...|
|brazilian man don...|        -0.0772|[brazilian, man, ...|
|absolutely last t...|        -0.4404|[absolutely, last...|
|would counter shi...|        -0.1779|[would, counter, ...|
+--------------------+---------------+--------------------+
only showing top 5 rows

In [77]:
word2Vec = Word2Vec(vectorSize=100, minCount=5, inputCol='words', outputCol='w2_vector')
w2v_model = word2Vec.fit(df_tokenized)
transform_w2v = w2v_model.transform(df_tokenized)
transform_w2v.select('w2_vector').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           w2_vector|
+--------------------+
|[-0.0012583285570...|
|[-0.0590399055956...|
|[0.00635056311805...|
|[-0.0522531430469...|
|[0.02936405253907...|
+--------------------+
only showing top 5 rows

### Most similar words to 'girl'

In [78]:
top_n = 10

sim_words_girl=w2v_model.findSynonyms('girl', top_n).select('word', fmt('similarity', 5).alias('similarity')).toPandas()
sim_words_girl.head(top_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         word similarity
0       chick    0.81825
1         guy    0.63390
2  girlfriend    0.56461
3       woman    0.54702
4         hb8    0.54700
5         gal    0.54121
6      blonde    0.53347
7      hotter    0.52709
8    sorority    0.51088
9        cute    0.51087

### Most similar words to 'woman'

In [81]:
sim_words_women=w2v_model.findSynonyms('woman', top_n).select('word', fmt('similarity', 5).alias('similarity')).toPandas()
sim_words_women.head(top_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

               word similarity
0               men    0.65025
1       objectified    0.58079
2              rele    0.56953
3              girl    0.54702
4          feminine    0.54008
5           loyalty    0.51931
6       promiscuous    0.51716
7       pedastalize    0.51647
8  husbandboyfriend    0.51427
9         desirable    0.51371

### Most similar words to 'feminist' (since many incels on this subreddit talk negatively about feminists)

In [82]:
sim_words_feminist=w2v_model.findSynonyms('feminist', top_n).select('word', fmt('similarity', 5).alias('similarity')).toPandas()
sim_words_feminist.head(top_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         word similarity
0         sjw    0.72513
1    feminism    0.67483
2        sjws    0.64657
3     radical    0.61928
4     liberal    0.61631
5      openly    0.61582
6        anti    0.61291
7   thirdwave    0.60567
8  misogynist    0.60542
9   supremacy    0.60440

### Most similar words to 'stacy' (word referring to 'perfect'/unobtainable woman)

In [83]:
sim_words_stacy=w2v_model.findSynonyms('stacy', top_n).select('word', fmt('similarity', 5).alias('similarity')).toPandas()
sim_words_stacy.head(top_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          word similarity
0    obedience    0.72934
1      whorish    0.71832
2    protector    0.71243
3       schlub    0.71231
4  committment    0.70744
5       maggot    0.70685
6        putty    0.70662
7      servile    0.70216
8    masochist    0.70129
9    misbehave    0.69837

### Most similar words to 'chad' (referring to perfect/alpha man)

In [84]:
sim_words_chad=w2v_model.findSynonyms('chad', top_n).select('word', fmt('similarity', 5).alias('similarity')).toPandas()
sim_words_chad.head(top_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

           word similarity
0   thundercock    0.76120
1         billy    0.66107
2       widowed    0.64209
3         alpha    0.64077
4         chump    0.63740
5            bb    0.62806
6       betabux    0.62020
7         widow    0.61716
8  thundercocks    0.61616
9        tyrone    0.61528

### Most similar words to 'becky' (woman that's the opposite of a stacy)

In [91]:
sim_words_becky=w2v_model.findSynonyms('becky', top_n).select('word', fmt('similarity', 5).alias('similarity')).toPandas()
sim_words_becky.head(top_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          word similarity
0     thanking    0.74624
1          711    0.74476
2       buster    0.74387
3  coordinator    0.74126
4      peacock    0.73053
5       shrill    0.72806
6     etcetera    0.72385
7          cab    0.72318
8        spree    0.72243
9         prod    0.71904

### Most similar words to 'beta' (since many men on this subreddit see themselves as betas, not alphas)

In [92]:
sim_words_beta=w2v_model.findSynonyms('beta', top_n).select('word', fmt('similarity', 5).alias('similarity')).toPandas()
sim_words_beta.head(top_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        word similarity
0      alpha    0.80729
1        bux    0.76561
2        fux    0.74707
3      omega    0.70937
4  fucksbeta    0.70718
5       cuck    0.67084
6      widow    0.67001
7       buck    0.66542
8    orbiter    0.66092
9   provider    0.65838

### Most similar words to 'love' (since many incels on this subreddit are seeking a girlfriend)

In [87]:
sim_words_love=w2v_model.findSynonyms('love', top_n).select('word', fmt('similarity', 5).alias('similarity')).toPandas()
sim_words_love.head(top_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              word similarity
0    unconditional    0.63432
1  unconditionally    0.59699
2            puppy    0.55327
3        heartless    0.48756
4         selflove    0.47339
5        homemaker    0.46907
6           loving    0.45918
7        beautiful    0.45756
8         kindness    0.45667
9         motherly    0.44484

### Most similar words to 'wish' (since many incels on this subreddit feel inadequate)

In [88]:
sim_words_wish=w2v_model.findSynonyms('wish', top_n).select('word', fmt('similarity', 5).alias('similarity')).toPandas()
sim_words_wish.head(top_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         word similarity
0      upvote    0.49127
1      relate    0.47049
2        2001    0.43497
3      brotha    0.42316
4       loved    0.42003
5      hahaha    0.41684
6     glimmer    0.40989
7   elaborate    0.40873
8  motivating    0.40812
9        hope    0.40192

### Most similar words to 'deserve' (since many incels on this subreddit feel that they are entitled to women)

In [90]:
sim_words_deserve=w2v_model.findSynonyms('deserve', top_n).select('word', fmt('similarity', 5).alias('similarity')).toPandas()
sim_words_deserve.head(top_n)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         word similarity
0   homemaker    0.59974
1         owe    0.58529
2       whine    0.55778
3   heartless    0.53394
4      coddle    0.53095
5    bitching    0.51546
6      fooled    0.50647
7         shh    0.50047
8        care    0.50014
9  moralistic    0.49990